In [1]:
with open("./api_key.txt", "r") as f:
    api_key = f.readline()
    
with open("./tavily_key.txt", "r") as f:
    tavily_key = f.readline()

In [7]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langgraph.checkpoint.sqlite import SqliteSaver

In [3]:
import os

os.environ["TAVILY_API_KEY"] = tavily_key

In [4]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [14]:
class Agent:

    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_chatgroq)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_chatgroq(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [15]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatGroq(
    model="llama3-8b-8192",  # Ensure this model ID is correct
    temperature=0.7,
    api_key=api_key
)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [16]:
messages = [HumanMessage(content='What is the weather in Bangalore?')]

In [17]:
thread = {"configurable": {"thread_id": "1"}}

In [24]:
from langgraph.checkpoint.sqlite import SqliteSaver

with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)
    messages = [HumanMessage(content='What is the weather in Patna?')]
    thread = {"configurable": {"thread_id": "1"}}
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7r8m', 'function': {'arguments': '{"query":"weather in Patna"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2051, 'total_tokens': 2124, 'completion_time': 0.060833333, 'prompt_time': 0.552128645, 'queue_time': -0.640774873, 'total_time': 0.612961978}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9b0252b9-1baa-4698-a6a9-7a4460684df0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_7r8m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2051, 'output_tokens': 73, 'total_tokens': 2124})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_7r8m', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.weat

With same thread id, we are able to provide context to the model

In [23]:
from langgraph.checkpoint.sqlite import SqliteSaver

with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)
    messages = [HumanMessage(content='What is the weather in Patna?')]
    thread = {"configurable": {"thread_id": "1"}}
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])
    messages = [HumanMessage(content='What is the weather in Bangalore?')]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])
    messages = [HumanMessage(content='Which one is warmer?')]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mfw5', 'function': {'arguments': '{"query":"weather in Patna"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 2051, 'total_tokens': 2124, 'completion_time': 0.060833333, 'prompt_time': 0.298645829, 'queue_time': -0.38761834799999995, 'total_time': 0.359479162}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-32a305ac-f476-4d79-b601-1790ecf0a93b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_mfw5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2051, 'output_tokens': 73, 'total_tokens': 2124})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_mfw5', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://

Lets change the thread id

In [25]:
from langgraph.checkpoint.sqlite import SqliteSaver

with SqliteSaver.from_conn_string(":memory:") as memory:
    abot = Agent(model, [tool], system=prompt, checkpointer=memory)
    messages = [HumanMessage(content='What is the weather in Patna?')]
    thread = {"configurable": {"thread_id": "1"}}
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])
    messages = [HumanMessage(content='What is the weather in Bangalore?')]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])
    thread = {"configurable": {"thread_id": "2"}}
    messages = [HumanMessage(content='Which one is warmer?')]
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_q14p', 'function': {'arguments': '{"query":"weather in Patna"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 1015, 'total_tokens': 1095, 'completion_time': 0.066666667, 'prompt_time': 0.058401862, 'queue_time': 0.0014832780000000045, 'total_time': 0.125068529}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b9e2e1a0-fef0-49d5-96c8-4eac98177520-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_q14p', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1015, 'output_tokens': 80, 'total_tokens': 1095})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Patna'}, 'id': 'call_q14p', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https:/

As we see, it loses the context